In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import operator
import pandas as pd
import matplotlib.pyplot as plt

from crycompare import *

%matplotlib inline

In [2]:
p = Price()
coinList = p.coinList()
coins = sorted(list( coinList['Data'].keys() ))

In [3]:
# Top 20 crypto by market cap
large_crypto = ['BTC', 'ETH', 'XRP', 'BCH', 'ADA', 'LTC', 'XEM', 'XLM', 'IOT', 'NEO', 'EOS', 'DASH', 'XMR', 'TRX', 'BTG', 'ETC', 'ICX', 'QTUM', 'LSK', 'XRB']

In [4]:
h = History()

df_dict = {}
for coin in coins:
    if coin in large_crypto:
        histo = h.histoDay(coin,'USD',allData=True)
        if histo['Data']:
            df_histo = pd.DataFrame(histo['Data'])
            df_histo['time'] = pd.to_datetime(df_histo['time'],unit='s')
            df_histo.index = df_histo['time']
            del df_histo['time']
            del df_histo['volumefrom']
            del df_histo['volumeto']

            df_dict[coin] = df_histo

In [5]:
crypto_histo = pd.concat(df_dict.values(), axis=1, keys=df_dict.keys())

In [6]:
histo_coins = [elem for elem in crypto_histo.columns.levels[0]]

In [7]:
histo_length = {}
for coin in histo_coins:
    histo_length[coin] = np.sum( ~np.isnan(crypto_histo[coin]['close'].values) )
    
sorted_length = sorted(histo_length.items(), key=operator.itemgetter(1),reverse=True)

In [8]:
# we keep the 10 coins having the longest time series of historical prices
sub_coins = [sorted_length[i][0] for i in range(10)]

sub_crypto_histo = crypto_histo[sub_coins]

In [9]:
N = len(sub_coins)
recent_histo = sub_crypto_histo[-200:]

In [10]:
recent_histo.head()

BTC                               LTC                       \
              close     high      low     open  close   high    low   open   
time                                                                         
2017-06-29  2558.37  2605.86  2526.51  2577.74  40.13  41.94  39.45  41.89   
2017-06-30  2480.61  2576.28  2469.54  2558.37  39.48  41.22  39.29  40.13   
2017-07-01  2424.61  2529.62  2387.47  2480.61  38.08  39.70  37.12  39.48   
2017-07-02  2536.46  2555.34  2375.86  2424.61  41.26  41.65  36.87  38.08   
2017-07-03  2572.47  2617.48  2506.30  2536.46  45.34  47.56  40.36  41.26   

              DASH           ...        XLM            LSK                    \
             close    high   ...        low     open close  high   low  open   
time                         ...                                               
2017-06-29  172.13  181.59   ...    0.02784  0.03149  2.84  3.02  2.71  2.80   
2017-06-30  173.70  179.39   ...    0.02635  0.02834  2.59  3.00  2.50  2.84   
2017-07-01  162.01  177.54   ...    0.02238  0.02637  2.47  2.67  2.31  2.59   
2017-07-02  169.78  171.37   ...    0.02170  0.02394  2.56  2.67  2.21  2.47   
2017-07-03  178.55  181.05   ...    0.02411  0.02523  2.42  2.66  2.29  2.56   

               XEM                          
             close    high     low    open  
time                                        
2017-06-29  0.1587  0.1754  0.1554  0.1692  
2017-06-30  0.1559  0.1665  0.1550  0.1587  
2017-07-01  0.1415  0.1583  0.1380  0.1559  
2017-07-02  0.1561  0.1568  0.1394  0.1415  
2017-07-03  0.1735  0.1781  0.1516  0.1561  

[5 rows x 40 columns]

In [11]:
ohlc_dict = {}
for coin in sub_coins:
    df_individual = recent_histo[coin].copy()
    df_individual['Average'] = df_individual.mean(numeric_only=True, axis=1)
    
    ohlc_dict[coin] = df_individual

In [12]:
crypto_ohlc = pd.concat(ohlc_dict.values(), axis=1, keys=ohlc_dict.keys())

In [13]:
crypto_ohlc.head()

BTC                                          LTC         \
              close     high      low     open    Average  close   high   
time                                                                      
2017-06-29  2558.37  2605.86  2526.51  2577.74  2567.1200  40.13  41.94   
2017-06-30  2480.61  2576.28  2469.54  2558.37  2521.2000  39.48  41.22   
2017-07-01  2424.61  2529.62  2387.47  2480.61  2455.5775  38.08  39.70   
2017-07-02  2536.46  2555.34  2375.86  2424.61  2473.0675  41.26  41.65   
2017-07-03  2572.47  2617.48  2506.30  2536.46  2558.1775  45.34  47.56   

                                     ...      LSK                            \
              low   open  Average    ...    close  high   low  open Average   
time                                 ...                                      
2017-06-29  39.45  41.89  40.8525    ...     2.84  3.02  2.71  2.80  2.8425   
2017-06-30  39.29  40.13  40.0300    ...     2.59  3.00  2.50  2.84  2.7325   
2017-07-01  37.12  39.48  38.5950    ...     2.47  2.67  2.31  2.59  2.5100   
2017-07-02  36.87  38.08  39.4650    ...     2.56  2.67  2.21  2.47  2.4775   
2017-07-03  40.36  41.26  43.6300    ...     2.42  2.66  2.29  2.56  2.4825   

               XEM                                    
             close    high     low    open   Average  
time                                                  
2017-06-29  0.1587  0.1754  0.1554  0.1692  0.164675  
2017-06-30  0.1559  0.1665  0.1550  0.1587  0.159025  
2017-07-01  0.1415  0.1583  0.1380  0.1559  0.148425  
2017-07-02  0.1561  0.1568  0.1394  0.1415  0.148450  
2017-07-03  0.1735  0.1781  0.1516  0.1561  0.164825  

[5 rows x 50 columns]

In [14]:
returns_dict = {}
for coin in sub_coins:
    coin_histo = crypto_ohlc[coin]
    coin_returns = pd.DataFrame(np.diff(np.log(coin_histo['Average']),axis=0))
    returns_dict[coin] = coin_returns

recent_returns = pd.concat(returns_dict.values(),axis=1,keys=returns_dict.keys())

In [16]:
recent_returns.index = recent_histo.index[1:]

In [17]:
recent_returns.head()

,BTC,LTC,DASH,XRP,XMR,ETH,ETC,XLM,LSK,XEM
,0,0,0,0,0,0,0,0,0,0
time,,,,,,,,,,
2017-06-30,-0.018050,-0.020339,-0.012057,-0.037030,-0.043162,-0.057170,-0.031457,-0.087760,-0.039467,-0.034912
2017-07-01,-0.026373,-0.036506,-0.035624,-0.037011,-0.058475,-0.065289,-0.049420,-0.098713,-0.084934,-0.068982
2017-07-02,0.007097,0.022291,-0.014957,0.009535,-0.009397,0.014577,0.002522,-0.035611,-0.013033,0.000168
2017-07-03,0.033836,0.100331,0.049389,0.027377,0.040115,0.022509,0.027479,0.061046,0.002016,0.104636
2017-07-04,0.017995,0.136328,0.038707,-0.002386,0.041042,-0.022043,0.007183,0.026805,0.009023,0.085848


In [18]:
average_returns_df = recent_returns.mean()
std_dev_returns_df = recent_returns.std()

In [21]:
correlation_matrix_df = recent_returns.corr()

In [22]:
correlation_matrix_df

,,BTC,LTC,DASH,XRP,XMR,ETH,ETC,XLM,LSK,XEM
,,0,0,0,0,0,0,0,0,0,0
BTC,0,1.000000,0.468746,0.299012,0.149753,0.376018,0.441253,0.362970,0.303321,0.179997,0.335875
LTC,0,0.468746,1.000000,0.484304,0.326889,0.474352,0.640201,0.592759,0.317170,0.299640,0.476566
DASH,0,0.299012,0.484304,1.000000,0.240239,0.637670,0.569115,0.471048,0.272676,0.468614,0.410391
XRP,0,0.149753,0.326889,0.240239,1.000000,0.343828,0.460857,0.319931,0.454227,0.276235,0.371641
XMR,0,0.376018,0.474352,0.637670,0.343828,1.000000,0.581898,0.543784,0.435374,0.526011,0.431227
ETH,0,0.441253,0.640201,0.569115,0.460857,0.581898,1.000000,0.668014,0.410999,0.405261,0.523558
ETC,0,0.362970,0.592759,0.471048,0.319931,0.543784,0.668014,1.000000,0.463853,0.399409,0.398557
XLM,0,0.303321,0.317170,0.272676,0.454227,0.435374,0.410999,0.463853,1.000000,0.323656,0.446097
LSK,0,0.179997,0.299640,0.468614,0.276235,0.526011,0.405261,0.399409,0.323656,1.000000,0.318153
